#### Imports

In [1]:
import pandas as pd
from Bio import Entrez

In [ ]:
Entrez.email = "example@gmail.com"
date_range = '("2012/03/01"[Date - Create] : "2015/12/31"[Date - Create])'
Entrez.api_key = "api_key"

### Checking the number of records available for our query

In [ ]:
full_query = date_range
handle = Entrez.esearch(db = 'pubmed', retmax = 1000, term = full_query)
record = Entrez.read(handle)
id_list = record["IdList"]
print(record['Count'])

### Creating a pandas dataframe and inserting the records with their PMID, abstracts and the date of completion

In [ ]:
df = pd.DataFrame(columns = ["PMID", "Abstract", "Date"])

for pmid in id_list:
    handle = Entrez.efetch(db = "pubmed", id = pmid, retmode = "xml")
    records = Entrez.read(handle)
    
    for record in records["PubmedArticle"]:
        pmid = record["MedlineCitation"]["PMID"]

        if "Abstract" in record["MedlineCitation"]["Article"] and "AbstractText" in record["MedlineCitation"]["Article"]["Abstract"]:
            abstract = "".join(record["MedlineCitation"]["Article"]["Abstract"]["AbstractText"])
        else:
            abstract = ""
        if "DateCompleted" in record["MedlineCitation"]:
            date = record["MedlineCitation"]["DateCompleted"]["Year"]
        else:
            date = record["MedlineCitation"]["Article"]["Journal"]["JournalIssue"]["PubDate"]["Year"]

        new_row = pd.DataFrame({
            'PMID' : [pmid],
            "Abstract" : [abstract],
            "Date" : [date]
        })

        df = pd.concat([df, new_row], ignore_index = True)

### Converting the dataframe to excel file

In [ ]:
df.to_excel("pubmed_results.xlsx", index = False)